In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Cargar los datos
file_path = r'C:\Users\gcordero\Documents\Github\Data_Science_portfolio\Machine_Learning\Calendarizacion de Transportes\Consolidado de Transporte.txt'

# Leer el archivo con encoding latin-1 para manejar caracteres especiales
df = pd.read_csv(file_path, 
                 sep='\t', 
                 encoding='latin-1',
                 decimal=',',
                 thousands='.')

# Ver las primeras filas
print("Primeras filas del dataset:")
print(df.head())
print("\n" + "="*80 + "\n")

# Información general
print("Información del dataset:")
print(df.info())
print("\n" + "="*80 + "\n")

# Dimensiones
print(f"Dimensiones del dataset: {df.shape[0]} filas y {df.shape[1]} columnas")

Primeras filas del dataset:
  Transportista Población Origen  Destino Fecha Expedicion  \
0           PDQ            LAMPA   Osorno       08-10-2025   
1           PDQ            LAMPA  Iquique       14-10-2025   
2           PDQ         SANTIAGO  Iquique       14-10-2025   
3           PDQ         SANTIAGO  Iquique       14-10-2025   
4           PDQ     PUERTO MONTT    Matta       14-10-2025   

   Periodo Facturacion  Volumen   Peso  Importe   
0                45962    0.840  210.0    33.600  
1                45962    0.041   12.0     4.200  
2                45962    0.007    5.0     4.200  
3                45962    0.139   35.0     7.350  
4                45962    0.675  169.0    14.365  


Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12079 entries, 0 to 12078
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Transportista        12079 non-null  object 
 1   Pob

In [27]:
# Limpiar nombres de columnas (eliminar espacios)
df.columns = df.columns.str.strip()

# Función para limpiar y convertir importes
def limpiar_importe(valor):
    if pd.isna(valor):
        return np.nan
    # Remover $, espacios y convertir
    valor_limpio = str(valor).replace('$', '').replace('.', '').replace(' ', '').strip()
    try:
        return float(valor_limpio)
    except:
        return np.nan

# Aplicar limpieza a la columna Importe
df['Importe'] = df['Importe'].apply(limpiar_importe)

# Convertir Fecha Expedicion a datetime (robusto)
# Guardar copia del valor original para depuración
df['Fecha_Expedicion_original'] = df['Fecha Expedicion']

# Intentar parsear con dayfirst=True (maneja 'DD-MM-YYYY' y formatos similares)
df['Fecha Expedicion'] = pd.to_datetime(
    df['Fecha Expedicion'],
    dayfirst=True,
    infer_datetime_format=True,
    errors='coerce'
)

# Informar si hubo valores no parseados y mostrar ejemplos
n_unparseadas = df['Fecha Expedicion'].isna().sum()
print(f"Fechas no parseadas: {n_unparseadas}")
if n_unparseadas > 0:
    print("Ejemplos de valores de fecha que no se pudieron convertir:")
    print(df.loc[df['Fecha Expedicion'].isna(), 'Fecha_Expedicion_original'].unique()[:20])
# Verificar valores nulos
print("Valores nulos por columna:")
print(df.isnull().sum())
print("\n" + "="*80 + "\n")

# Estadísticas descriptivas
print("Estadísticas descriptivas de variables numéricas:")
print(df[['Volumen', 'Peso', 'Importe']].describe())
print("\n" + "="*80 + "\n")

# Verificar el rango de fechas
print(f"Rango de fechas: {df['Fecha Expedicion'].min()} a {df['Fecha Expedicion'].max()}")
print(f"Periodo de análisis: {(df['Fecha Expedicion'].max() - df['Fecha Expedicion'].min()).days} días")

Fechas no parseadas: 0
Valores nulos por columna:
Transportista                 0
Población Origen              0
Destino                       0
Fecha Expedicion              0
Periodo Facturacion           0
Volumen                       6
Peso                         28
Importe                      31
Fecha_Expedicion_original     0
dtype: int64


Estadísticas descriptivas de variables numéricas:
            Volumen          Peso       Importe
count  12073.000000  12051.000000  1.204800e+04
mean       0.910300    267.434535  4.132760e+08
std        2.548582    716.803079  1.026490e+09
min        0.000000      0.000000  1.900000e+06
25%        0.031000     10.250000  5.104000e+07
50%        0.172000     53.000000  1.226200e+08
75%        0.868000    252.000000  3.668650e+08
max       76.076883  19250.000000  2.729294e+10


Rango de fechas: 2025-05-29 00:00:00 a 2025-11-20 00:00:00
Periodo de análisis: 175 días


In [28]:
# Análisis de Transportistas
print("=" * 80)
print("ANÁLISIS DE TRANSPORTISTAS")
print("=" * 80)
print("\nTransportistas únicos:")
print(df['Transportista'].value_counts())
print("\nTotal de transportistas diferentes:", df['Transportista'].nunique())

print("\n" + "=" * 80)
print("ANÁLISIS DE DESTINOS")
print("=" * 80)
print("\nTop 20 Destinos más frecuentes:")
print(df['Destino'].value_counts().head(20))
print("\nTotal de destinos diferentes:", df['Destino'].nunique())

print("\n" + "=" * 80)
print("ANÁLISIS DE ORÍGENES")
print("=" * 80)
print("\nOrígenes más frecuentes:")
print(df['Población Origen'].value_counts())
print("\nTotal de orígenes diferentes:", df['Población Origen'].nunique())

print("\n" + "=" * 80)
print("ANÁLISIS DE COSTOS")
print("=" * 80)
# Filtrar registros con importe válido
df_valid = df[df['Importe'] > 1].copy()
print(f"\nRegistros con importe válido: {len(df_valid)} de {len(df)}")
print(f"Costo total del periodo: ${df_valid['Importe'].sum():,.0f}")
print(f"Costo promedio por envío: ${df_valid['Importe'].mean():,.0f}")
print(f"Costo mediano por envío: ${df_valid['Importe'].median():,.0f}")

ANÁLISIS DE TRANSPORTISTAS

Transportistas únicos:
Transportista
PDQ      6248
SAMEX    3223
TVP      2608
Name: count, dtype: int64

Total de transportistas diferentes: 3

ANÁLISIS DE DESTINOS

Top 20 Destinos más frecuentes:
Destino
Antofagasta         1004
Concepcion           983
Matta                869
Temuco               859
Valdivia             857
Iquique              804
Calama               698
Talca                690
Montt                677
Copiapo              646
Serena               607
Los Angeles          537
Chillan              520
Rancagua             511
Puerto Montt         465
Osorno               443
Valparaiso           213
Curauma              191
Los Libertadores     165
Otro                  98
Name: count, dtype: int64

Total de destinos diferentes: 26

ANÁLISIS DE ORÍGENES

Orígenes más frecuentes:
Población Origen
SANTIAGO                                  2522
COLINA                                     840
DARTEL MATTA (Centro Costo)                696

In [5]:
# Crear columna de Ruta
df_valid['Ruta'] = df_valid['Población Origen'] + ' → ' + df_valid['Destino']

print("=" * 80)
print("TOP 20 RUTAS MÁS FRECUENTES")
print("=" * 80)
rutas_frecuencia = df_valid['Ruta'].value_counts().head(20)
print(rutas_frecuencia)

print("\n" + "=" * 80)
print("TOP 20 RUTAS MÁS COSTOSAS (COSTO TOTAL ACUMULADO)")
print("=" * 80)
rutas_costo_total = df_valid.groupby('Ruta')['Importe'].agg(['sum', 'count', 'mean']).sort_values('sum', ascending=False)
print(rutas_costo_total.head(20))

print("\n" + "=" * 80)
print("TOP 20 RUTAS CON MAYOR COSTO PROMEDIO POR ENVÍO")
print("=" * 80)
# Filtrar rutas con al menos 10 envíos para que sea representativo
rutas_min_10 = rutas_costo_total[rutas_costo_total['count'] >= 10].sort_values('mean', ascending=False)
print(rutas_min_10.head(20))

print("\n" + "=" * 80)
print("RESUMEN DE ANÁLISIS DE RUTAS")
print("=" * 80)
print(f"Total de rutas únicas: {df_valid['Ruta'].nunique()}")
print(f"Rutas que representan el 80% del costo total: ", end='')
# Calcular Pareto (80/20)
costo_acumulado = rutas_costo_total['sum'].sort_values(ascending=False).cumsum()
total_cost = rutas_costo_total['sum'].sum()
rutas_80 = (costo_acumulado <= total_cost * 0.8).sum()
print(rutas_80)
print(f"Rutas que representan el 80% de la frecuencia: ", end='')
freq_acumulada = rutas_frecuencia.sort_values(ascending=False).cumsum()
total_freq = rutas_frecuencia.sum()
rutas_80_freq = (freq_acumulada <= total_freq * 0.8).sum()
print(rutas_80_freq)

TOP 20 RUTAS MÁS FRECUENTES
Ruta
SANTIAGO → Antofagasta                                    275
SANTIAGO → Temuco                                         263
SANTIAGO → Valdivia                                       243
SANTIAGO → Iquique                                        220
SANTIAGO → Chillan                                        217
SANTIAGO → Calama                                         196
SANTIAGO → Los Angeles                                    189
SANTIAGO → Serena                                         183
SANTIAGO → Copiapo                                        174
COLINA → Antofagasta                                      159
COLINA → Iquique                                          158
DARTEL MATTA (Centro Costo) → Puerto Montt                144
DARTEL MATTA (Centro Costo) → Talca                       138
DARTEL MATTA (Centro Costo) → Concepcion                  132
COLINA → Serena                                           126
COLINA → Calama                      

In [29]:
# Normalizacion Columna de Origen
print("=" * 80)
print("NORMALIZACIÓN DE ORÍGENES")
print("=" * 80)

# Ver todos los orígenes únicos para identificar cuáles normalizar
print("\nTodos los orígenes únicos:")
origenes_unicos = sorted(df_valid['Población Origen'].unique())
for i, origen in enumerate(origenes_unicos, 1):
    print(f"{i}. {origen}")

print("\n" + "=" * 80)
print("APLICANDO NORMALIZACIÓN")
print("=" * 80)

# Función para normalizar orígenes
def normalizar_origen(origen):
    origen_upper = str(origen).upper()
    
    # Todo lo que contenga estos términos es SANTIAGO
    terminos_santiago = ['SANTIAGO', 'LAMPA', 'COLINA', 'QUILICURA', 'RENCA', 
                         'PUDAHUEL', 'VITACURA', 'MATTA', 'LOS LIBERTADORES',
                         'LO RUIZ', 'RUIZ', 'DARTEL']
    
    for termino in terminos_santiago:
        if termino in origen_upper:
            return 'SANTIAGO'
    
    # Valparaíso
    if 'CURAUMA' in origen_upper or 'VALPARAISO' in origen_upper:
        return 'VALPARAISO'
    
    # Si no coincide con nada, mantener el original (normalizado a mayúsculas)
    return origen.upper()

# Aplicar normalización
df_valid['Origen_Normalizado'] = df_valid['Población Origen'].apply(normalizar_origen)

# Rehacer la columna Ruta con origen normalizado
df_valid['Ruta'] = df_valid['Origen_Normalizado'] + ' → ' + df_valid['Destino']

print("\nOrígenes ANTES de normalizar:")
print(df['Población Origen'].value_counts().head(20))

print("\n" + "=" * 80)
print("Orígenes DESPUÉS de normalizar:")
print(df_valid['Origen_Normalizado'].value_counts())

print("\n" + "=" * 80)
print("NUEVAS TOP 20 RUTAS MÁS FRECUENTES (NORMALIZADAS)")
print("=" * 80)
rutas_frecuencia = df_valid['Ruta'].value_counts().head(20)
print(rutas_frecuencia)

print("\n" + "=" * 80)
print("NUEVAS TOP 20 RUTAS MÁS COSTOSAS (NORMALIZADAS)")
print("=" * 80)
rutas_costo_total = df_valid.groupby('Ruta')['Importe'].agg(['sum', 'count', 'mean']).sort_values('sum', ascending=False)
print(rutas_costo_total.head(20))

print("\n" + "=" * 80)
print("RESUMEN ACTUALIZADO")
print("=" * 80)
print(f"Total de rutas únicas (normalizadas): {df_valid['Ruta'].nunique()}")
costo_acumulado = rutas_costo_total['sum'].sort_values(ascending=False).cumsum()
total_cost = rutas_costo_total['sum'].sum()
rutas_80 = (costo_acumulado <= total_cost * 0.8).sum()
print(f"Rutas que representan el 80% del costo total: {rutas_80}")

NORMALIZACIÓN DE ORÍGENES

Todos los orígenes únicos:
1. 3M CHILE S.A.
2. ANDES CONSULTORIA Y REPRESENTACION SPA
3. ANTOFAGASTA
4. CALAMA
5. CD Los Ruizcentros (centro de Costo)
6. CHILLAN
7. COBRE CERRILLOS S.A.
8. COLINA
9. COM SERV ACTION IMPORT LTDA
10. COMERCIAL E INVERSIONES COMATEL S.A.
11. COMERCIAL MERY SPA
12. COMERCIAL METALCONEX LTDA
13. COMERCIALIZADORA VALDES SPA
14. CONCEPCION
15. COPIAPO
16. COQUIMBO
17. COYHAIQUE
18. CURAUMA
19. CURICO
20. DARCO LTDA
21. DARTEL CD LOS LIBERTADORES (Centro Costo)
22. DARTEL CHILLAN (Centro Costo)
23. DARTEL CONCEPCION (Centro Costo)
24. DARTEL CURAUMA (Centro Costo)
25. DARTEL LA SERENA (Centro Costo)
26. DARTEL LIRA (Centro de Costo)
27. DARTEL LOS ANGELES (Centro Costo)
28. DARTEL MATTA (Centro Costo)
29. DARTEL OSORNO (Centro de costo )
30. DARTEL PUERTO MONTT (Centro Costo)
31. DARTEL RANCAGUA ( Centro Costo)
32. DARTEL S.A
33. DARTEL TALCA (Centro Costo)
34. DARTEL TEMUCO (Centro Costo)
35. DARTEL VALDIVIA (Centro Costo)
36. DARTEL

In [30]:
# Agregar columnas de tiempo
df_valid['Mes'] = df_valid['Fecha Expedicion'].dt.to_period('M')
df_valid['Semana'] = df_valid['Fecha Expedicion'].dt.to_period('W')
df_valid['Dia_Semana'] = df_valid['Fecha Expedicion'].dt.day_name()

print("=" * 80)
print("ANÁLISIS DE FRECUENCIA TEMPORAL")
print("=" * 80)

print("\nEnvíos por mes:")
envios_mes = df_valid.groupby('Mes').agg({
    'Importe': ['sum', 'count', 'mean']
}).round(0)
envios_mes.columns = ['Costo Total', 'Num Envíos', 'Costo Promedio']
print(envios_mes)
print(f"\nPromedio mensual: {envios_mes['Num Envíos'].mean():.0f} envíos, ${envios_mes['Costo Total'].mean():,.0f}")

print("\n" + "=" * 80)
print("FRECUENCIA POR DÍA DE LA SEMANA")
print("=" * 80)
dias_orden = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
envios_dia = df_valid.groupby('Dia_Semana').agg({
    'Importe': ['sum', 'count', 'mean']
}).reindex(dias_orden)
envios_dia.columns = ['Costo Total', 'Num Envíos', 'Costo Promedio']
print(envios_dia)

print("\n" + "=" * 80)
print("ANÁLISIS DE FRECUENCIA PARA TOP 10 RUTAS MÁS COSTOSAS")
print("=" * 80)
top_10_rutas = rutas_costo_total.head(10).index
for ruta in top_10_rutas:
    df_ruta = df_valid[df_valid['Ruta'] == ruta]
    dias_entre_envios = df_ruta['Fecha Expedicion'].sort_values().diff().dt.days.dropna()
    
    print(f"\n{ruta}")
    print(f"  Total envíos: {len(df_ruta)}")
    print(f"  Costo total: ${df_ruta['Importe'].sum():,.0f}")
    print(f"  Frecuencia promedio: cada {dias_entre_envios.mean():.1f} días")
    print(f"  Frecuencia mediana: cada {dias_entre_envios.median():.1f} días")
    print(f"  Envíos/mes estimados: {30/dias_entre_envios.mean():.1f}")
    print(f"  Peso promedio: {df_ruta['Peso'].mean():.1f} kg")
    print(f"  Volumen promedio: {df_ruta['Volumen'].mean():.2f} m³")

ANÁLISIS DE FRECUENCIA TEMPORAL

Envíos por mes:
          Costo Total  Num Envíos  Costo Promedio
Mes                                              
2025-05  4.416000e+07           1      44160000.0
2025-06  6.568540e+09          14     469181429.0
2025-07  1.158867e+10          17     681686471.0
2025-08  7.506619e+11        1689     444441628.0
2025-09  1.462537e+12        3760     388972633.0
2025-10  1.795873e+12        4094     438659751.0
2025-11  9.518752e+11        2473     384907097.0

Promedio mensual: 1721 envíos, $711,306,950,000

FRECUENCIA POR DÍA DE LA SEMANA
             Costo Total  Num Envíos  Costo Promedio
Dia_Semana                                          
Monday      9.956678e+11      2373.0    4.195819e+08
Tuesday     1.116240e+12      2676.0    4.171301e+08
Wednesday   1.017434e+12      2391.0    4.255267e+08
Thursday    1.015496e+12      2556.0    3.972989e+08
Friday      8.161321e+11      2026.0    4.028293e+08
Saturday    1.817819e+10        26.0    6.991612

In [31]:
print("=" * 80)
print("FILTRADO DE DATOS - PERIODO REAL DE ANÁLISIS")
print("=" * 80)

# Filtrar usando los objetos Period correctos
meses_principales = [pd.Period('2025-09', 'M'), pd.Period('2025-10', 'M'), pd.Period('2025-11', 'M')]
df_valid_periodo = df_valid[df_valid['Mes'].isin(meses_principales)].copy()

print(f"\nRegistros totales antes del filtro: {len(df_valid)}")
print(f"Registros en periodo real (Sep-Nov 2025): {len(df_valid_periodo)}")
print(f"Registros descartados (rezagos): {len(df_valid) - len(df_valid_periodo)}")

print("\n" + "=" * 80)
print("RESUMEN DEL PERIODO REAL (SEP-NOV 2025)")
print("=" * 80)

resumen_periodo = df_valid_periodo.groupby('Mes').agg({
    'Importe': ['sum', 'count', 'mean']
}).round(0)
resumen_periodo.columns = ['Costo Total', 'Num Envíos', 'Costo Promedio']
print(resumen_periodo)
print(f"\n{'='*80}")
print(f"TOTALES DEL PERIODO (3 MESES):")
print(f"  Total envíos: {len(df_valid_periodo):,}")
print(f"  Costo total: ${df_valid_periodo['Importe'].sum():,.0f}")
print(f"  Promedio mensual: {len(df_valid_periodo)/3:.0f} envíos")
print(f"  Costo promedio mensual: ${df_valid_periodo['Importe'].sum()/3:,.0f}")
print(f"  Costo promedio por envío: ${df_valid_periodo['Importe'].mean():,.0f}")

# Actualizar dataset de trabajo
df_valid = df_valid_periodo.copy()

print("\n" + "=" * 80)
print("TOP 15 RUTAS MÁS COSTOSAS (PERIODO REAL)")
print("=" * 80)
rutas_costo_total = df_valid.groupby('Ruta')['Importe'].agg(['sum', 'count', 'mean']).sort_values('sum', ascending=False)
print(rutas_costo_total.head(15))

costo_acumulado = rutas_costo_total['sum'].sort_values(ascending=False).cumsum()
total_cost = rutas_costo_total['sum'].sum()
rutas_80 = (costo_acumulado <= total_cost * 0.8).sum()
print(f"\nRutas que representan el 80% del costo: {rutas_80}")

FILTRADO DE DATOS - PERIODO REAL DE ANÁLISIS

Registros totales antes del filtro: 12048
Registros en periodo real (Sep-Nov 2025): 10327
Registros descartados (rezagos): 1721

RESUMEN DEL PERIODO REAL (SEP-NOV 2025)
          Costo Total  Num Envíos  Costo Promedio
Mes                                              
2025-09  1.462537e+12        3760     388972633.0
2025-10  1.795873e+12        4094     438659751.0
2025-11  9.518752e+11        2473     384907097.0

TOTALES DEL PERIODO (3 MESES):
  Total envíos: 10,327
  Costo total: $4,210,285,370,000
  Promedio mensual: 3442 envíos
  Costo promedio mensual: $1,403,428,456,667
  Costo promedio por envío: $407,696,850

TOP 15 RUTAS MÁS COSTOSAS (PERIODO REAL)
                                    sum  count          mean
Ruta                                                        
SANTIAGO → Antofagasta     4.553691e+11    608  7.489624e+08
SANTIAGO → Iquique         3.649149e+11    480  7.602393e+08
SANTIAGO → Puerto Montt    2.232818e+11   

In [32]:
print("=" * 80)
print("ANÁLISIS DE COSTOS POR TRANSPORTISTA")
print("=" * 80)

# Análisis general por transportista
transportista_stats = df_valid.groupby('Transportista').agg({
    'Importe': ['sum', 'count', 'mean', 'median'],
    'Peso': 'sum',
    'Volumen': 'sum'
}).round(0)
transportista_stats.columns = ['Costo Total', 'Num Envíos', 'Costo Promedio', 'Costo Mediano', 'Peso Total (kg)', 'Volumen Total (m³)']
print(transportista_stats)

print("\n" + "=" * 80)
print("COSTO POR KG Y M³ POR TRANSPORTISTA")
print("=" * 80)
transportista_stats['$/kg'] = (transportista_stats['Costo Total'] / transportista_stats['Peso Total (kg)']).round(0)
transportista_stats['$/m³'] = (transportista_stats['Costo Total'] / transportista_stats['Volumen Total (m³)']).round(0)
print(transportista_stats[['Num Envíos', 'Costo Promedio', '$/kg', '$/m³']])

print("\n" + "=" * 80)
print("TOP 5 RUTAS: COMPARACIÓN DE COSTOS POR TRANSPORTISTA")
print("=" * 80)

top_5_rutas = rutas_costo_total.head(5).index

for ruta in top_5_rutas:
    print(f"\n{ruta}:")
    df_ruta = df_valid[df_valid['Ruta'] == ruta]
    comparacion = df_ruta.groupby('Transportista').agg({
        'Importe': ['count', 'mean', 'median', 'sum'],
        'Peso': 'mean',
        'Volumen': 'mean'
    }).round(0)
    comparacion.columns = ['Envíos', 'Costo Promedio', 'Costo Mediano', 'Costo Total', 'Peso Prom (kg)', 'Vol Prom (m³)']
    if len(comparacion) > 0:
        print(comparacion.sort_values('Costo Promedio', ascending=False))

ANÁLISIS DE COSTOS POR TRANSPORTISTA
                Costo Total  Num Envíos  Costo Promedio  Costo Mediano  \
Transportista                                                            
PDQ            1.303976e+12        5365     243052352.0     74160000.0   
SAMEX          1.729617e+12        2703     639887969.0    180920000.0   
TVP            1.176692e+12        2259     520890801.0    235710000.0   

               Peso Total (kg)  Volumen Total (m³)  
Transportista                                       
PDQ                   778985.0              2543.0  
SAMEX                 998969.0              3996.0  
TVP                   949585.0              2732.0  

COSTO POR KG Y M³ POR TRANSPORTISTA
               Num Envíos  Costo Promedio       $/kg         $/m³
Transportista                                                    
PDQ                  5365     243052352.0  1673942.0  512770692.0
SAMEX                2703     639887969.0  1731402.0  432837132.0
TVP                  2259 

In [15]:
print("=" * 80)
print("ANÁLISIS DETALLADO: ¿POR QUÉ PDQ ES TAN CARO?")
print("=" * 80)

# Comparar características de envíos por transportista
print("\nCARACTERÍSTICAS PROMEDIO POR TRANSPORTISTA:")
caracteristicas = df_valid.groupby('Transportista').agg({
    'Peso': ['mean', 'median', 'min', 'max'],
    'Volumen': ['mean', 'median', 'min', 'max'],
    'Importe': ['mean', 'median']
}).round(2)
print(caracteristicas)

print("\n" + "=" * 80)
print("DISTRIBUCIÓN DE TAMAÑO DE ENVÍOS")
print("=" * 80)

# Crear categorías de tamaño
def categorizar_envio(row):
    peso = row['Peso']
    if peso <= 50:
        return 'Muy Pequeño (<50kg)'
    elif peso <= 200:
        return 'Pequeño (50-200kg)'
    elif peso <= 500:
        return 'Mediano (200-500kg)'
    elif peso <= 1000:
        return 'Grande (500-1000kg)'
    else:
        return 'Muy Grande (>1000kg)'

df_valid['Categoria_Tamano'] = df_valid.apply(categorizar_envio, axis=1)

# Ver distribución por transportista
dist_tamano = pd.crosstab(df_valid['Transportista'], 
                           df_valid['Categoria_Tamano'], 
                           normalize='index') * 100
print("\nDistribución porcentual por tamaño de envío:")
print(dist_tamano.round(1))

print("\n" + "=" * 80)
print("COSTO PROMEDIO POR CATEGORÍA DE TAMAÑO")
print("=" * 80)
costo_por_tamano = df_valid.groupby(['Transportista', 'Categoria_Tamano'])['Importe'].agg(['count', 'mean']).round(0)
costo_por_tamano.columns = ['Cantidad', 'Costo Promedio']
print(costo_por_tamano)

print("\n" + "=" * 80)
print("HIPÓTESIS: ¿PDQ cobra por envío en vez de por peso/volumen?")
print("=" * 80)
# Correlación entre peso/volumen y costo por transportista
for transp in ['PDQ', 'SAMEX', 'TVP']:
    df_t = df_valid[df_valid['Transportista'] == transp]
    corr_peso = df_t[['Peso', 'Importe']].corr().iloc[0, 1]
    corr_vol = df_t[['Volumen', 'Importe']].corr().iloc[0, 1]
    print(f"\n{transp}:")
    print(f"  Correlación Peso-Costo: {corr_peso:.3f}")
    print(f"  Correlación Volumen-Costo: {corr_vol:.3f}")
    print(f"  Interpretación: {'Cobra más por peso/volumen' if corr_peso > 0.7 else 'Cobra por envío o servicio'}")

ANÁLISIS DETALLADO: ¿POR QUÉ PDQ ES TAN CARO?

CARACTERÍSTICAS PROMEDIO POR TRANSPORTISTA:
                 Peso                         Volumen                      \
                 mean  median   min       max    mean median   min    max   
Transportista                                                               
PDQ            159.68   25.00  1.00  18000.00    0.52   0.08  0.00  72.00   
SAMEX          369.85   90.00  0.25  16800.00    1.48   0.36  0.00  67.20   
TVP            421.66  189.12  2.02  19019.22    1.21   0.60  0.01  76.08   

                  Importe            
                     mean    median  
Transportista                        
PDQ            1115762.51  299152.0  
SAMEX            63988.80   18092.0  
TVP              52089.08   23571.0  

DISTRIBUCIÓN DE TAMAÑO DE ENVÍOS

Distribución porcentual por tamaño de envío:
Categoria_Tamano  Grande (500-1000kg)  Mediano (200-500kg)  \
Transportista                                                
PDQ           

In [34]:
print("=" * 80)
print("VERIFICACIÓN DE DATOS - MUESTRA DE IMPORTES ORIGINALES")
print("=" * 80)

# Recargar el archivo original sin procesar
df_original = pd.read_csv(file_path, 
                          sep='\t', 
                          encoding='latin-1',
                          nrows=20)

print("\nNombres de columnas en archivo original:")
print(df_original.columns.tolist())

print("\n" + "=" * 80)
print("Primeras 20 filas SIN procesar:")
print(df_original.head(20))

# Limpiar nombres de columnas
df_original.columns = df_original.columns.str.strip()

print("\n" + "=" * 80)
print("Después de limpiar nombres de columnas:")
print(df_original.columns.tolist())

print("\n" + "=" * 80)
print("Muestra de datos relevantes:")
if 'Importe' in df_original.columns:
    print(df_original[['Transportista', 'Destino', 'Peso', 'Importe']].head(20))
else:
    print("Todas las columnas disponibles:")
    print(df_original.head(20))

VERIFICACIÓN DE DATOS - MUESTRA DE IMPORTES ORIGINALES

Nombres de columnas en archivo original:
['Transportista', 'Población Origen', 'Destino', 'Fecha Expedicion', 'Periodo Facturacion', 'Volumen', 'Peso', ' Importe ']

Primeras 20 filas SIN procesar:
   Transportista Población Origen           Destino Fecha Expedicion  \
0            PDQ            LAMPA            Osorno       2025-10-08   
1            PDQ            LAMPA           Iquique       2025-10-14   
2            PDQ         SANTIAGO           Iquique       2025-10-14   
3            PDQ         SANTIAGO           Iquique       2025-10-14   
4            PDQ     PUERTO MONTT             Matta       2025-10-14   
5            PDQ     PUERTO MONTT             Matta       2025-10-14   
6            PDQ         SANTIAGO           Iquique       2025-10-15   
7            PDQ       CONCEPCION  Los Libertadores       2025-10-15   
8            PDQ       CONCEPCION        Concepcion       2025-10-15   
9            PDQ          

In [35]:
print("=" * 80)
print("DIAGNÓSTICO: COMPARANDO DATOS CARGADOS VS ORIGINALES")
print("=" * 80)

# Ver los primeros 20 registros del dataframe que ya tenemos cargado
print("\nPrimeros 20 importes en df_valid (ya procesado):")
print(df_valid[['Transportista', 'Destino', 'Peso', 'Importe']].head(20))

print("\n" + "=" * 80)
print("Estadísticas de Importe en df_valid:")
print(df_valid['Importe'].describe())

print("\n" + "=" * 80)
print("Pregunta: ¿El archivo que estamos usando es el correcto?")
print(f"Archivo cargado: {file_path}")
print("\n¿Este archivo tiene los datos de flete CORREGIDOS que mencionaste?")
print("Por favor confirma y si es necesario proporciona la ruta del archivo actualizado.")

DIAGNÓSTICO: COMPARANDO DATOS CARGADOS VS ORIGINALES

Primeros 20 importes en df_valid (ya procesado):
   Transportista           Destino   Peso      Importe
0            PDQ            Osorno  210.0  336000000.0
1            PDQ           Iquique   12.0   42000000.0
2            PDQ           Iquique    5.0   42000000.0
3            PDQ           Iquique   35.0   73500000.0
4            PDQ             Matta  169.0  143650000.0
5            PDQ             Matta    8.0   61000000.0
6            PDQ           Iquique   26.0   54600000.0
7            PDQ  Los Libertadores   16.0   43000000.0
8            PDQ        Concepcion   64.0  194560000.0
9            PDQ       Antofagasta    2.0   37000000.0
10           PDQ       Antofagasta   55.0  101750000.0
11           PDQ       Antofagasta    5.0   37000000.0
12           PDQ           Iquique  300.0  429000000.0
13           PDQ            Calama   15.0   79050000.0
14           PDQ       Antofagasta  180.0  457200000.0
15           PDQ 

In [36]:
print("=" * 80)
print("RECARGA COMPLETA DE DATOS CON CONVERSIÓN CORRECTA")
print("=" * 80)

# Leer el archivo SIN conversión automática de decimales
df = pd.read_csv(file_path, 
                 sep='\t', 
                 encoding='latin-1')

# Limpiar nombres de columnas
df.columns = df.columns.str.strip()

# Función CORREGIDA para limpiar importes (trabajar con el texto original)
def limpiar_importe_correcto(valor):
    if pd.isna(valor):
        return np.nan
    # Convertir a string y limpiar
    valor_str = str(valor).replace('$', '').replace(' ', '').strip()
    
    # Reemplazar punto por nada (es separador de miles en formato chileno)
    # Reemplazar coma por punto (es separador decimal)
    valor_str = valor_str.replace('.', '').replace(',', '.')
    
    try:
        return float(valor_str)
    except:
        return np.nan

# Aplicar limpieza
df['Importe'] = df['Importe'].apply(limpiar_importe_correcto)
df['Volumen'] = pd.to_numeric(df['Volumen'].astype(str).str.replace(',', '.'), errors='coerce')
df['Peso'] = pd.to_numeric(df['Peso'], errors='coerce')

# Convertir fechas
df['Fecha Expedicion'] = pd.to_datetime(df['Fecha Expedicion'], format='%Y-%m-%d', errors='coerce')

print("\nPrimeras 20 filas con importes CORREGIDOS:")
print(df[['Transportista', 'Destino', 'Peso', 'Importe']].head(20))

print("\n" + "=" * 80)
print("Estadísticas de Importe CORREGIDO:")
print(df['Importe'].describe())

print("\n¿Estos valores se ven correctos ahora?")

RECARGA COMPLETA DE DATOS CON CONVERSIÓN CORRECTA

Primeras 20 filas con importes CORREGIDOS:
   Transportista           Destino   Peso  Importe
0            PDQ            Osorno  210.0  33600.0
1            PDQ           Iquique   12.0   4200.0
2            PDQ           Iquique    5.0   4200.0
3            PDQ           Iquique   35.0   7350.0
4            PDQ             Matta  169.0  14365.0
5            PDQ             Matta    8.0   6100.0
6            PDQ           Iquique   26.0   5460.0
7            PDQ  Los Libertadores   16.0   4300.0
8            PDQ        Concepcion   64.0  19456.0
9            PDQ       Antofagasta    2.0   3700.0
10           PDQ       Antofagasta   55.0  10175.0
11           PDQ       Antofagasta    5.0   3700.0
12           PDQ           Iquique  300.0  42900.0
13           PDQ            Calama   15.0   7905.0
14           PDQ       Antofagasta  180.0  45720.0
15           PDQ       Antofagasta   16.0   3700.0
16           PDQ   CD Libertadores    1

In [37]:
print("=" * 80)
print("PREPROCESAMIENTO COMPLETO CON DATOS CORRECTOS")
print("=" * 80)

# Normalizar orígenes
def normalizar_origen(origen):
    origen_upper = str(origen).upper()
    terminos_santiago = ['SANTIAGO', 'LAMPA', 'COLINA', 'QUILICURA', 'RENCA', 
                         'PUDAHUEL', 'VITACURA', 'MATTA', 'LOS LIBERTADORES',
                         'LO RUIZ', 'RUIZ', 'DARTEL']
    for termino in terminos_santiago:
        if termino in origen_upper:
            return 'SANTIAGO'
    if 'CURAUMA' in origen_upper or 'VALPARAISO' in origen_upper:
        return 'VALPARAISO'
    return origen.upper()

# Aplicar transformaciones
df['Origen_Normalizado'] = df['Población Origen'].apply(normalizar_origen)
df['Ruta'] = df['Origen_Normalizado'] + ' → ' + df['Destino']
df['Mes'] = df['Fecha Expedicion'].dt.to_period('M')

# Filtrar datos válidos (importes > 1 para eliminar errores)
df_valid = df[df['Importe'] > 1].copy()

# Filtrar solo el periodo real (Sep-Nov 2025)
meses_principales = [pd.Period('2025-09', 'M'), pd.Period('2025-10', 'M'), pd.Period('2025-11', 'M')]
df_valid = df_valid[df_valid['Mes'].isin(meses_principales)].copy()

print(f"\nRegistros válidos en periodo Sep-Nov 2025: {len(df_valid):,}")
print(f"Costo total 3 meses: ${df_valid['Importe'].sum():,.0f}")
print(f"Costo promedio por envío: ${df_valid['Importe'].mean():,.0f}")
print(f"Promedio mensual: {len(df_valid)/3:.0f} envíos, ${df_valid['Importe'].sum()/3:,.0f}")

print("\n" + "=" * 80)
print("RESUMEN POR TRANSPORTISTA")
print("=" * 80)
resumen_transp = df_valid.groupby('Transportista').agg({
    'Importe': ['sum', 'count', 'mean'],
    'Peso': 'sum',
    'Volumen': 'sum'
})
resumen_transp.columns = ['Costo Total', 'Envíos', 'Costo Prom', 'Peso Total (kg)', 'Vol Total (m³)']
resumen_transp['$/kg'] = (resumen_transp['Costo Total'] / resumen_transp['Peso Total (kg)']).round(0)
print(resumen_transp)

print("\n" + "=" * 80)
print("TOP 15 RUTAS MÁS COSTOSAS")
print("=" * 80)
rutas_costo = df_valid.groupby('Ruta')['Importe'].agg(['sum', 'count', 'mean']).sort_values('sum', ascending=False)
print(rutas_costo.head(15))

PREPROCESAMIENTO COMPLETO CON DATOS CORRECTOS

Registros válidos en periodo Sep-Nov 2025: 10,327
Costo total 3 meses: $421,028,537
Costo promedio por envío: $40,770
Promedio mensual: 3442 envíos, $140,342,846

RESUMEN POR TRANSPORTISTA
               Costo Total  Envíos    Costo Prom  Peso Total (kg)  \
Transportista                                                       
PDQ            130397587.0    5365  24305.235228         778985.0   
SAMEX          172961718.0    2703  63988.796892         895903.0   
TVP            117669232.0    2259  52089.080124          22312.0   

               Vol Total (m³)    $/kg  
Transportista                          
PDQ               2543.132000   167.0  
SAMEX             3995.874760   193.0  
TVP               2732.416196  5274.0  

TOP 15 RUTAS MÁS COSTOSAS
                                  sum  count          mean
Ruta                                                      
SANTIAGO → Antofagasta     45536912.0    608  74896.236842
SANTIAGO → Iqu

In [40]:
print("=" * 80)
print("NORMALIZACIÓN DE DESTINOS")
print("=" * 80)

print("\nDestinos únicos ANTES de normalizar:")
print(df_valid['Destino'].value_counts().head(30))

# Función para normalizar destinos
def normalizar_destino(destino):
    destino_upper = str(destino).upper().strip()
    
    # Normalizar Puerto Montt
    if 'MONTT' in destino_upper and 'PUERTO' not in destino_upper:
        return 'Puerto Montt'
    
    # Normalizar CD Libertadores
    if 'LIBERTADORES' in destino_upper or 'LOS LIBERTADORES' in destino_upper:
        return 'CD Libertadores'
    
    # Capitalizar correctamente destinos comunes
    normalizaciones = {
        'ANTOFAGASTA': 'Antofagasta',
        'IQUIQUE': 'Iquique',
        'CALAMA': 'Calama',
        'COPIAPO': 'Copiapo',
        'SERENA': 'Serena',
        'VALPARAISO': 'Valparaiso',
        'RANCAGUA': 'Rancagua',
        'TALCA': 'Talca',
        'CHILLAN': 'Chillan',
        'CONCEPCION': 'Concepcion',
        'TEMUCO': 'Temuco',
        'VALDIVIA': 'Valdivia',
        'OSORNO': 'Osorno',
        'PUERTO MONTT': 'Puerto Montt',
        'LOS ANGELES': 'Los Angeles',
        'CURAUMA': 'Curauma',
        'MATTA': 'Matta',
        'COQUIMBO': 'Coquimbo'
    }
    
    return normalizaciones.get(destino_upper, destino.title())

# Aplicar normalización
df_valid['Destino'] = df_valid['Destino'].apply(normalizar_destino)

# Recrear la columna Ruta con destino normalizado
df_valid['Ruta'] = df_valid['Origen_Normalizado'] + ' → ' + df_valid['Destino']

print("\n" + "=" * 80)
print("Destinos únicos DESPUÉS de normalizar:")
print(df_valid['Destino'].value_counts().head(30))

print("\n" + "=" * 80)
print("TOP 15 RUTAS ACTUALIZADAS (con normalización)")
print("=" * 80)
rutas_costo = df_valid.groupby('Ruta')['Importe'].agg(['sum', 'count', 'mean']).sort_values('sum', ascending=False)
print(rutas_costo.head(15))

NORMALIZACIÓN DE DESTINOS

Destinos únicos ANTES de normalizar:
Destino
Antofagasta            859
Concepcion             844
Matta                  756
Valdivia               747
Temuco                 735
Iquique                664
Talca                  610
Calama                 589
Montt                  587
Copiapo                550
Serena                 518
Los Angeles            481
Chillan                442
Rancagua               416
Puerto Montt           397
Osorno                 385
Valparaiso             179
Curauma                158
Los Libertadores       125
CD Libertadores         85
Otro                    77
Vitacura                45
Lira                    39
CD Los Libertadores     28
CD Lo Ruiz               9
La Serena                2
Name: count, dtype: int64

Destinos únicos DESPUÉS de normalizar:
Destino
Puerto Montt       984
Antofagasta        859
Concepcion         844
Matta              756
Valdivia           747
Temuco             735
Iquique       

In [43]:
print("=" * 80)
print("CORRECCIÓN FINAL: NORMALIZACIÓN VALPARAÍSO Y RUTAS OPTIMIZADAS")
print("=" * 80)

# Normalizar Curauma como Valparaíso
df_valid.loc[df_valid['Destino'] == 'Curauma', 'Destino'] = 'Valparaiso'
df_valid['Ruta'] = df_valid['Origen_Normalizado'] + ' → ' + df_valid['Destino']

# Recalcular desde Santiago
df_santiago = df_valid[df_valid['Origen_Normalizado'] == 'SANTIAGO'].copy()

# Calcular volúmenes SEMANALES
volumenes_semanales = df_santiago.groupby('Destino').agg({
    'Volumen': 'sum',
    'Peso': 'sum',
    'Importe': ['sum', 'count']
})
volumenes_semanales.columns = ['Vol Total', 'Peso Total', 'Costo Total', 'Envíos']
volumenes_semanales['Vol/semana (m³)'] = (volumenes_semanales['Vol Total'] / 12).round(1)
volumenes_semanales['Envíos/semana'] = (volumenes_semanales['Envíos'] / 12).round(1)
volumenes_semanales['% Ocupación'] = (volumenes_semanales['Vol/semana (m³)'] / 50 * 100).round(1)
volumenes_semanales['Costo/semana'] = (volumenes_semanales['Costo Total'] / 12).round(0)
volumenes_semanales = volumenes_semanales.sort_values('Vol/semana (m³)', ascending=False)

print("\nVOLÚMENES SEMANALES ACTUALIZADOS (Top 20):")
print("-" * 80)
print(f"{'Destino':<20} {'m³/sem':<10} {'%Ocup':<10} {'Envíos/sem':<12} {'Costo/sem'}")
print("-" * 80)
for dest, row in volumenes_semanales.head(20).iterrows():
    print(f"{dest:<20} {row['Vol/semana (m³)']:<10.1f} {row['% Ocupación']:<10.1f} {row['Envíos/semana']:<12.1f} ${row['Costo/semana']:,.0f}")

print("\n" + "=" * 80)
print("PROPUESTA FINAL DE RUTAS (BASADA EN TU FEEDBACK)")
print("=" * 80)

# Definir rutas finales
rutas_finales = [
    {
        'nombre': 'Ruta 1A: Santiago → Antofagasta (DIRECTO)',
        'destinos': ['Antofagasta'],
        'vol_estimado': 50.0,
        'frecuencia': 'Semanal',
        'camiones': 1,
        'notas': 'Carga completa directa'
    },
    {
        'nombre': 'Ruta 1B: Santiago → Antofagasta → Calama (COMPARTIDO)',
        'destinos': ['Antofagasta', 'Calama'],
        'vol_estimado': None,  # Calcular
        'frecuencia': 'Semanal',
        'camiones': 2,
        'notas': 'Antofagasta residual (37 m³) + Calama (38 m³) = ~75 m³ = 2 camiones'
    },
    {
        'nombre': 'Ruta 2: Santiago → Iquique (DIRECTO)',
        'destinos': ['Iquique'],
        'vol_estimado': None,
        'frecuencia': 'Semanal',
        'camiones': 1,
        'notas': 'Carga completa'
    },
    {
        'nombre': 'Ruta 3: Santiago → Serena (DIRECTO)',
        'destinos': ['Serena'],
        'vol_estimado': None,
        'frecuencia': 'Semanal',
        'camiones': 1,
        'notas': '~49 m³/semana - casi lleno'
    },
    {
        'nombre': 'Ruta 4: Santiago → Copiapo (DIRECTO)',
        'destinos': ['Copiapo'],
        'vol_estimado': None,
        'frecuencia': 'Semanal',
        'camiones': 1,
        'notas': '~41 m³/semana - 82% ocupación'
    },
    {
        'nombre': 'Ruta 5: Santiago → Puerto Montt (DIRECTO)',
        'destinos': ['Puerto Montt'],
        'vol_estimado': None,
        'frecuencia': 'Semanal',
        'camiones': 1,
        'notas': '~42 m³/semana - 84% ocupación'
    },
    {
        'nombre': 'Ruta 6: Santiago → Talca → Chillan',
        'destinos': ['Talca', 'Chillan'],
        'vol_estimado': None,
        'frecuencia': 'Semanal',
        'camiones': 1,
        'notas': 'Talca de paso hacia Chillán'
    },
    {
        'nombre': 'Ruta 7: Santiago → Los Angeles → Concepcion',
        'destinos': ['Los Angeles', 'Concepcion'],
        'vol_estimado': None,
        'frecuencia': 'Semanal',
        'camiones': 1,
        'notas': 'Ruta natural Sur'
    },
    {
        'nombre': 'Ruta 8: Santiago → Temuco → Valdivia',
        'destinos': ['Temuco', 'Valdivia'],
        'vol_estimado': None,
        'frecuencia': 'Semanal',
        'camiones': 1,
        'notas': 'Costa Sur consolidada'
    },
    {
        'nombre': 'Ruta 9: Santiago → Osorno → Puerto Montt (Alternativa)',
        'destinos': ['Osorno', 'Puerto Montt'],
        'vol_estimado': None,
        'frecuencia': 'Según necesidad',
        'camiones': 1,
        'notas': 'Si Puerto Montt no llena camión solo'
    },
    {
        'nombre': 'Ruta 10: Santiago → Rancagua + Valparaiso (LOCAL)',
        'destinos': ['Rancagua', 'Valparaiso'],
        'vol_estimado': None,
        'frecuencia': 'Semanal o Quincenal',
        'camiones': 1,
        'notas': 'Rutas cortas metropolitanas'
    }
]

# Calcular volúmenes reales para cada ruta
print("\nDETALLE DE RUTAS PROPUESTAS:")
print("=" * 80)

for i, ruta in enumerate(rutas_finales, 1):
    print(f"\n{ruta['nombre']}")
    print("-" * 80)
    
    vol_total = 0
    for dest in ruta['destinos']:
        if dest in volumenes_semanales.index:
            vol = volumenes_semanales.loc[dest, 'Vol/semana (m³)']
            envios = volumenes_semanales.loc[dest, 'Envíos/semana']
            costo = volumenes_semanales.loc[dest, 'Costo/semana']
            vol_total += vol
            print(f"  {dest:<20} {vol:>6.1f} m³/sem  |  {envios:>5.1f} envíos/sem  |  ${costo:>10,.0f}/sem")
    
    if len(ruta['destinos']) > 1:
        print(f"  {'TOTAL RUTA':<20} {vol_total:>6.1f} m³/sem  |  Ocupación: {vol_total/50*100:.1f}%")
    
    print(f"  Frecuencia: {ruta['frecuencia']}")
    print(f"  Camiones/despacho: {ruta['camiones']}")
    print(f"  Nota: {ruta['notas']}")

print("\n" + "=" * 80)
print("RESUMEN EJECUTIVO")
print("=" * 80)
total_camiones = sum([r['camiones'] for r in rutas_finales[:8]])  # Rutas principales
print(f"\nTotal de camiones en operación semanal: {total_camiones}")
print(f"Costo actual total semanal: ${df_santiago['Importe'].sum() / 12:,.0f}")

CORRECCIÓN FINAL: NORMALIZACIÓN VALPARAÍSO Y RUTAS OPTIMIZADAS

VOLÚMENES SEMANALES ACTUALIZADOS (Top 20):
--------------------------------------------------------------------------------
Destino              m³/sem     %Ocup      Envíos/sem   Costo/sem
--------------------------------------------------------------------------------
Antofagasta          87.2       174.4      50.7         $3,794,743
Iquique              60.0       120.0      40.0         $3,040,957
Serena               48.7       97.4       33.5         $1,573,717
Puerto Montt         42.2       84.4       43.8         $2,328,403
Copiapo              41.1       82.2       32.9         $1,651,712
Calama               37.6       75.2       34.8         $1,740,194
Talca                37.2       74.4       35.1         $1,310,868
Temuco               35.6       71.2       32.8         $1,821,033
Concepcion           34.4       68.8       39.6         $1,447,679
Valdivia             24.0       48.0       25.3         $1,316

In [52]:
print("=" * 80)
print("CALENDARIO SEMANAL DE DESPACHOS - VERSIÓN COMPLETA CON OSORNO")
print("=" * 80)

# Calendario completo con Osorno incluido
calendario_despachos = {
    'Lunes': [
        {
            'ruta': 'Ruta 1B: Santiago → Antofagasta → Calama',
            'camiones': 1,
            'destinos': ['Antofagasta', 'Calama'],
            'vol_m3': 50.0,
            'vol_total': 75.0,
            'excedente': 25.0,
            'distancia_km': 1558,
            'costo_semanal': (3794743 / 2) + 1740194,
            'nota': '1 camión lleno + 25m³ excedente por courier. Entrega mar/mié'
        },
        {
            'ruta': 'Ruta 10: Santiago → Rancagua → Valparaiso',
            'camiones': 1,
            'destinos': ['Rancagua', 'Valparaiso'],
            'vol_m3': 22.5,
            'vol_total': 22.5,
            'excedente': 0,
            'distancia_km': 120,
            'costo_semanal': 537337 + 127130,
            'nota': 'Rutas locales cortas (45% ocupación). Entrega lunes mismo día'
        }
    ],
    'Martes': [
        {
            'ruta': 'Ruta 5: Santiago → Osorno → Puerto Montt',
            'camiones': 1,
            'destinos': ['Osorno', 'Puerto Montt'],
            'vol_m3': 49.6,
            'vol_total': 49.6,
            'excedente': 0,
            'distancia_km': 1016,
            'costo_semanal': 476652 + 2328403,
            'nota': '1 camión (99% ocupación - ÓPTIMO). Osorno de paso. Entrega miércoles'
        },
        {
            'ruta': 'Ruta 6: Santiago → Talca → Chillan',
            'camiones': 1,
            'destinos': ['Talca', 'Chillan'],
            'vol_m3': 50.0,
            'vol_total': 59.1,
            'excedente': 9.1,
            'distancia_km': 407,
            'costo_semanal': 1310868 + 849308,
            'nota': '1 camión lleno + 9m³ excedente por courier. Entrega martes'
        }
    ],
    'Miércoles': [
        {
            'ruta': 'Ruta 8: Santiago → Temuco → Valdivia',
            'camiones': 1,
            'destinos': ['Temuco', 'Valdivia'],
            'vol_m3': 50.0,
            'vol_total': 59.6,
            'excedente': 9.6,
            'distancia_km': 840,
            'costo_semanal': 1821033 + 1316624,
            'nota': '1 camión lleno + 10m³ excedente por courier. Entrega jueves'
        },
        {
            'ruta': 'Ruta 7: Santiago → Los Angeles → Concepcion',
            'camiones': 1,
            'destinos': ['Los Angeles', 'Concepcion'],
            'vol_m3': 46.2,
            'vol_total': 46.2,
            'excedente': 0,
            'distancia_km': 515,
            'costo_semanal': 509660 + 1447679,
            'nota': '1 camión (92% ocupación). Entrega miércoles/jueves'
        }
    ],
    'Jueves': [
        {
            'ruta': 'Ruta 3: Santiago → Serena',
            'camiones': 1,
            'destinos': ['Serena'],
            'vol_m3': 48.7,
            'vol_total': 48.7,
            'excedente': 0,
            'distancia_km': 471,
            'costo_semanal': 1573717,
            'nota': '1 camión (97% ocupación). Entrega jueves/viernes'
        },
        {
            'ruta': 'Ruta 4: Santiago → Copiapo',
            'camiones': 1,
            'destinos': ['Copiapo'],
            'vol_m3': 41.1,
            'vol_total': 41.1,
            'excedente': 0,
            'distancia_km': 804,
            'costo_semanal': 1651712,
            'nota': '1 camión (82% ocupación). Entrega viernes'
        }
    ],
    'Viernes': [
        {
            'ruta': 'Ruta 2: Santiago → Iquique (VIAJE FIN DE SEMANA)',
            'camiones': 1,
            'destinos': ['Iquique'],
            'vol_m3': 50.0,
            'vol_total': 60.0,
            'excedente': 10.0,
            'distancia_km': 1849,
            'costo_semanal': 3040957,
            'nota': '🚚 Sale viernes, viaja FDS, entrega LUNES. 1 camión lleno + 10m³ courier'
        },
        {
            'ruta': 'Ruta 1A: Santiago → Antofagasta DIRECTO (VIAJE FIN DE SEMANA)',
            'camiones': 1,
            'destinos': ['Antofagasta'],
            'vol_m3': 50.0,
            'vol_total': 87.2,
            'excedente': 37.2,
            'distancia_km': 1356,
            'costo_semanal': 3794743 / 2,
            'nota': '🚚 Sale viernes, viaja FDS, entrega LUNES. El resto va lunes compartido'
        }
    ]
}

# Mostrar calendario
print("\n📅 CALENDARIO SEMANAL DE DESPACHOS")
print("=" * 80)

total_camiones_semana = 0
total_costo_semana = 0
total_vol_flota = 0
total_excedente = 0
uso_rampas_por_dia = {}

dias_laborales = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']

for dia, despachos in calendario_despachos.items():
    print(f"\n{'🟢 ' + dia.upper():=^80}")
    
    camiones_dia = sum([d['camiones'] for d in despachos])
    uso_rampas_por_dia[dia] = camiones_dia
    
    for despacho in despachos:
        total_camiones_semana += despacho['camiones']
        total_costo_semana += despacho['costo_semanal']
        total_vol_flota += despacho['vol_m3']
        total_excedente += despacho['excedente']
        
        print(f"\n  {despacho['ruta']}")
        print(f"  {'─' * 76}")
        print(f"  Destinos: {' → '.join(despacho['destinos'])}")
        print(f"  Distancia: {despacho['distancia_km']} km")
        print(f"  Volumen: {despacho['vol_m3']:.1f} m³ de {despacho['vol_total']:.1f} m³ totales ({despacho['vol_m3']/50*100:.0f}% ocupación)", end='')
        if despacho['excedente'] > 0:
            print(f"\n           ⚠️  Excedente: {despacho['excedente']:.1f} m³ por courier")
        else:
            print()
        print(f"  Costo actual/semana: ${despacho['costo_semanal']:,.0f}")
        print(f"  📝 {despacho['nota']}")

print("\n" + "=" * 80)
print("RESUMEN SEMANAL")
print("=" * 80)
print(f"\nTotal de camiones flota propia/semana: {total_camiones_semana} despachos")
print(f"Volumen transportado flota propia: {total_vol_flota:.1f} m³/semana ({total_vol_flota/50:.1f} camiones llenos)")
print(f"Volumen total generado: {total_vol_flota + total_excedente:.1f} m³/semana")
print(f"Excedente por courier: {total_excedente:.1f} m³/semana ({total_excedente/(total_vol_flota + total_excedente)*100:.1f}%)")
print(f"\nCosto actual total/semana: ${total_costo_semana:,.0f}")
print(f"Costo actual mensual (4 semanas): ${total_costo_semana * 4:,.0f}")
print(f"Costo actual anual proyectado: ${total_costo_semana * 52:,.0f}")

print("\n" + "=" * 80)
print("USO DE RAMPAS POR DÍA")
print("=" * 80)
print(f"\n{'Día':<15} {'Camiones':<12} {'Estado':<40}")
print("-" * 80)
for dia in dias_laborales:
    camiones = uso_rampas_por_dia.get(dia, 0)
    if camiones == 2:
        indicador = "🟢🟢 COMPLETO (2 rampas usadas)"
    elif camiones == 1:
        indicador = "🟢⚪ PARCIAL (1 rampa libre)"
    else:
        indicador = "⚪⚪ LIBRE (2 rampas disponibles)"
    print(f"{dia:<15} {camiones:<12} {indicador}")

print("\n" + "=" * 80)
print("VISUALIZACIÓN SEMANAL CON TODAS LAS RUTAS")
print("=" * 80)
print("""
        LUNES       MARTES      MIÉRCOLES   JUEVES      VIERNES
    ┌──────────┐ ┌──────────┐ ┌──────────┐ ┌──────────┐ ┌──────────┐
R1  │Antofa.+  │ │ Osorno + │ │  Temuco  │ │  Serena  │ │ Iquique  │
    │  Calama  │ │P. Montt  │ │    +     │ │          │ │ (FDS)    │
    │  50m³*   │ │  50m³    │ │ Valdivia │ │  49m³    │ │  50m³*   │
    │          │ │(99% llen)│ │  50m³*   │ │          │ │          │
    ├──────────┤ ├──────────┤ ├──────────┤ ├──────────┤ ├──────────┤
R2  │ Rancagua │ │  Talca   │ │Los Ang.+ │ │ Copiapo  │ │Antofagas.│
    │    +     │ │    +     │ │Concepción│ │          │ │ DIRECTO  │
    │Valparaíso│ │ Chillan  │ │  46m³    │ │  41m³    │ │ (FDS)    │
    │  23m³    │ │  50m³*   │ │          │ │          │ │  50m³*   │
    └──────────┘ └──────────┘ └──────────┘ └──────────┘ └──────────┘
      2 rampas    2 rampas     2 rampas    2 rampas     2 rampas

    * = Camión lleno (50m³), excedente por courier
    FDS = Viaja Fin De Semana (sábado/domingo), entrega Lunes
""")

print("\n💡 NOTAS OPERACIONALES:")
print("  ✓ Total: 10 camiones/semana usando 2 rampas")
print("  ✓ TODOS los días usan 2 rampas (100% balanceado)")
print("  ✓ Ocupación promedio por camión: {:.0f}%".format(total_vol_flota/(total_camiones_semana*50)*100))
print("  ✓ Estrategia: Llenar camiones 100%, excedente por courier/PDQ")
print("  ✓ Excedente: {:.1f} m³/semana ({:.1f}% del total)".format(total_excedente, total_excedente/(total_vol_flota + total_excedente)*100))
print("  ✓ MARTES: Ruta Osorno→Puerto Montt consolidada (99% ocupación - ÓPTIMA)")
print("  ✓ VIERNES: Rutas extremas viajan fin de semana, entregan LUNES")
print("  ✓ Cobertura: Todas las rutas principales incluidas")

CALENDARIO SEMANAL DE DESPACHOS - VERSIÓN COMPLETA CON OSORNO

📅 CALENDARIO SEMANAL DE DESPACHOS

====================================🟢 LUNES=====================================

  Ruta 1B: Santiago → Antofagasta → Calama
  ────────────────────────────────────────────────────────────────────────────
  Destinos: Antofagasta → Calama
  Distancia: 1558 km
  Volumen: 50.0 m³ de 75.0 m³ totales (100% ocupación)
           ⚠️  Excedente: 25.0 m³ por courier
  Costo actual/semana: $3,637,566
  📝 1 camión lleno + 25m³ excedente por courier. Entrega mar/mié

  Ruta 10: Santiago → Rancagua → Valparaiso
  ────────────────────────────────────────────────────────────────────────────
  Destinos: Rancagua → Valparaiso
  Distancia: 120 km
  Volumen: 22.5 m³ de 22.5 m³ totales (45% ocupación)
  Costo actual/semana: $664,467
  📝 Rutas locales cortas (45% ocupación). Entrega lunes mismo día

====================================🟢 MARTES====================================

  Ruta 5: Santiago → Osorno → P